In [1]:
import findspark
findspark.init('/home/gani/spark-2.4.4-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructField, StructType, StringType, DoubleType

In [3]:
schema = StructType([
    StructField('geo',StringType(), True),
    StructField('yr1980', DoubleType(), False),
    StructField('yr1981', DoubleType(), False),
    StructField('yr1982', DoubleType(), False),
    StructField('yr1983', DoubleType(), False),
    StructField('yr1984', DoubleType(), False),
    StructField('yr1985', DoubleType(), False),
    StructField('yr1986', DoubleType(), False),
    StructField('yr1987', DoubleType(), False),
    StructField('yr1988', DoubleType(), False),
    StructField('yr1989', DoubleType(), False),
    StructField('yr1990', DoubleType(), False),
    StructField('yr1991', DoubleType(), False),
    StructField('yr1992', DoubleType(), False),
    StructField('yr1993', DoubleType(), False),
    StructField('yr1994', DoubleType(), False),
    StructField('yr1995', DoubleType(), False),
    StructField('yr1996', DoubleType(), False),
    StructField('yr1997', DoubleType(), False),
    StructField('yr1998', DoubleType(), False),
    StructField('yr1999', DoubleType(), False),
    StructField('yr2000', DoubleType(), False),
    StructField('yr2001', DoubleType(), False),
    StructField('yr2002', DoubleType(), False),
    StructField('yr2003', DoubleType(), False),
    StructField('yr2004', DoubleType(), False),
    StructField('yr2005', DoubleType(), False),
    StructField('yr2006', DoubleType(), False),
    StructField('yr2007', DoubleType(), False),
    StructField('yr2008', DoubleType(), False),
    StructField('yr2009', DoubleType(), False),
    StructField('yr2010', DoubleType(), False)
])

In [4]:
spark = SparkSession.builder.appName('sqlApp').getOrCreate()

In [5]:
df = spark.read.csv('populationbycountry19802010millions.csv', inferSchema=True, header=True, schema=schema)

In [6]:
for i in range(1981,2010):
    df = df.drop(F.col('yr{}'.format(i)))

In [7]:
df.show(5)

+-------------+---------+---------+
|          geo|   yr1980|   yr2010|
+-------------+---------+---------+
|North America|320.27638|456.59331|
|      Bermuda|  0.05473|  0.06827|
|       Canada|  24.5933| 33.75974|
|    Greenland|  0.05021|  0.05764|
|       Mexico| 68.34748|112.46886|
+-------------+---------+---------+
only showing top 5 rows



In [9]:
df = df.withColumn('evolution',F.expr('round((yr2010 - yr1980) * 1000000)'))

In [10]:
df.show(5)

+-------------+---------+---------+-----------+
|          geo|   yr1980|   yr2010|  evolution|
+-------------+---------+---------+-----------+
|North America|320.27638|456.59331|1.3631693E8|
|      Bermuda|  0.05473|  0.06827|    13540.0|
|       Canada|  24.5933| 33.75974|  9166440.0|
|    Greenland|  0.05021|  0.05764|     7430.0|
|       Mexico| 68.34748|112.46886| 4.412138E7|
+-------------+---------+---------+-----------+
only showing top 5 rows



In [12]:
df.createOrReplaceTempView('geodata')

In [13]:
query1 = """
        select * from geodata
        where geo is not null and evolution <= 0
        order by evolution
        limit 25"""

In [14]:
negativeEvo = spark.sql(query1)

In [15]:
negativeEvo.show(5)

+--------------------+-------+-------+---------+
|                 geo| yr1980| yr2010|evolution|
+--------------------+-------+-------+---------+
|Saint Pierre and ...|0.00599|0.00594|    -50.0|
|            Dominica|0.07389|0.07281|  -1080.0|
|Netherlands Antilles|0.23244|0.22869|  -3750.0|
|        Cook Islands|0.01801|0.01149|  -6520.0|
|          Montserrat|0.01177|0.00512|  -6650.0|
+--------------------+-------+-------+---------+
only showing top 5 rows



In [16]:
query2 = """
        select * from geodata
        where geo is not null and evolution > 999999
        order by evolution desc
        limit 25"""

In [17]:
moreThanMill = spark.sql(query2)

In [20]:
moreThanMill.show(5)

+--------------+----------+----------+------------+
|           geo|    yr1980|    yr2010|   evolution|
+--------------+----------+----------+------------+
|         World|4451.32679|6853.01941|2.40169262E9|
|Asia & Oceania|2469.81743|3799.67028|1.32985285E9|
|        Africa| 478.96479|1015.47842| 5.3651363E8|
|         India|  684.8877|1173.10802| 4.8822032E8|
|         China| 984.73646|1330.14129| 3.4540483E8|
+--------------+----------+----------+------------+
only showing top 5 rows

